# YouTube Statistics:
    
YouTube (the world-famous video sharing website) maintains a list of the top trending videos on the platform. According to Variety magazine, “To determine the year’s top-trending videos, YouTube uses a combination of factors including measuring users interactions (number of views, shares, comments and likes). Note that they’re not the most-viewed videos overall for the calendar year”. Top performers on the YouTube trending list are music videos (such as the famously virile “Gangam Style”), celebrity and/or reality TV performances, and the random dude-with-a-camera viral videos that YouTube is well-known for.
This dataset is a daily record of the top trending YouTube videos.
Note that this dataset is a structurally improved version of this dataset.


In [20]:
import pandas as pd
from sqlalchemy import create_engine
import json

In [21]:
us_video_file = "Resources/USvideos.csv"
ca_video_file = "Resources/CAvideos.csv"
us_video_json_file = "Resources/US_category_id.json"
ca_video_json_file = "Resources/CA_category_id.json"

us_video_df = pd.read_csv(us_video_file)
ca_video_df = pd.read_csv(ca_video_file)
#us_video_json_df = pd.read_json(us_video_json_file)
#ca_video_json_df = pd.read_json(ca_video_json_file)

# add  country code as new dataset column
us_video_df['country'] = 'USA'
ca_video_df['country'] = 'CA'


In [22]:
# merge US and Canada videos dataframe
merge_video_df = [us_video_df, ca_video_df]
us_ca_video_df = pd.concat(merge_video_df)

# removed error videos from US and Canada
new_us_video_df = us_ca_video_df[us_ca_video_df.video_error_or_removed != True ]
# create new dataset with only columns we need
new_us_video_df = new_us_video_df[['video_id', 'title', 'category_id', 'views', 'likes', 'dislikes', 'comment_count', 'country']].copy()
new_us_video_df.head()

,video_id,title,category_id,views,likes,dislikes,comment_count,country
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,22,748374,57527,2966,15954,USA
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,24,2418783,97185,6146,12703,USA
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",23,3191434,146033,5339,8181,USA
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,24,343168,10172,666,2146,USA
4,d380meD0W0M,I Dare You: GOING BALD!?,24,2095731,132235,1989,17518,USA


In [23]:
# get the US Category json data
ids = []
titles = []
us_category_df = pd.DataFrame()
with open(us_video_json_file) as data_file:
   data = json.load(data_file)
#Extract IDs
   for i in range(0,len(data["items"])):
       ids.append(data["items"][i]["id"])
# Extract Titles
for i in range(0, len(data["items"])):
   titles.append(data["items"][i]["snippet"]["title"])
# Create dataset with catagory ids and category titles
us_category_df["category_ids"] = ids
us_category_df["category_titles"] = titles
us_category_df.head()

,category_ids,category_titles
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [24]:
# Create connection to local database
rds_connection_string = "postgres:postgres@127.0.0.1:5432/youtube_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [25]:
# get the tables names available on youtube_db database
engine.table_names()

['youtube_data', 'youtube_category']

In [31]:
# Insert merged US and Canada dataframe to youtube_data table
new_us_video_df.to_sql(name='youtube_data', con=engine, if_exists='append', index=False)

In [32]:
# get the data from youtube_data table
pd.read_sql_query('select * from youtube_data', con=engine).head()

,video_id,title,category_id,views,likes,dislikes,comment_count,country
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,22,748374,57527,2966,15954,USA
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,24,2418783,97185,6146,12703,USA
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",23,3191434,146033,5339,8181,USA
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,24,343168,10172,666,2146,USA
4,d380meD0W0M,I Dare You: GOING BALD!?,24,2095731,132235,1989,17518,USA


In [33]:
# insert US category json data to youtube_category table
us_category_df.to_sql(name='youtube_category', con=engine, if_exists='append', index=False)

In [34]:
# get the youtube_category table data from postgres database
pd.read_sql_query('select * from youtube_category', con=engine).head()

,category_ids,category_titles
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
